In [104]:
import pandas as pd
import xgboost as xgb
import numpy as np

In [105]:
spammer_order = "../feature.json"
raw_file = pd.read_json(open(spammer_order), orient='records')
raw_file.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1214054254,0,0,0,15,0.500000,0,0,0,1005,...,0,7,0,0,0.034592,0,1,30972.857143,8.239935e+08,no
1,2158838150,0,0,0,10,1.733333,0,0,0,1819,...,0,3,0,0,0.026582,0,0,33023.000000,7.491242e+08,no
2,1465038162,0,0,0,18,0.984615,0,0,0,3542,...,0,3,0,0,0.027306,0,2,36454.250000,6.229687e+08,no
3,5203407567,0,0,0,27,0.012920,0,0,0,340,...,0,19,0,0,0.031989,0,4,11511.346939,7.102748e+08,no
4,3944654540,0,0,0,7,0.204396,0,0,0,371,...,0,3,0,0,0.055030,0,38,26378.770833,7.995144e+08,yes


In [106]:
all_feat_list = ['figure_RRT', 'average_comm', 'figure_at_sigle', 'hashtag', 'reputation', 
                 'late_night_times', 'followee_num', 'follow_ratio', 'post_num', 'figure_at', 
                 'active_day_ratio', 'mention', 'day_interval_variance', 'figure_at_every', 
                 'test_similarity', 'text_similarity', 'url', 'time_interval_mean', 'time_interval_var']

idx_title = ['id'] + all_feat_list + ['is_spammer']
print(idx_title)
raw_file.rename(columns={i: idx_title[i] for i in range(21)}, inplace=True)

['id', 'figure_RRT', 'average_comm', 'figure_at_sigle', 'hashtag', 'reputation', 'late_night_times', 'followee_num', 'follow_ratio', 'post_num', 'figure_at', 'active_day_ratio', 'mention', 'day_interval_variance', 'figure_at_every', 'test_similarity', 'text_similarity', 'url', 'time_interval_mean', 'time_interval_var', 'is_spammer']


In [107]:
raw_file.head()

,id,figure_RRT,average_comm,figure_at_sigle,hashtag,reputation,late_night_times,followee_num,follow_ratio,post_num,...,active_day_ratio,mention,day_interval_variance,figure_at_every,test_similarity,text_similarity,url,time_interval_mean,time_interval_var,is_spammer
0,1214054254,0,0,0,15,0.500000,0,0,0,1005,...,0,7,0,0,0.034592,0,1,30972.857143,8.239935e+08,no
1,2158838150,0,0,0,10,1.733333,0,0,0,1819,...,0,3,0,0,0.026582,0,0,33023.000000,7.491242e+08,no
2,1465038162,0,0,0,18,0.984615,0,0,0,3542,...,0,3,0,0,0.027306,0,2,36454.250000,6.229687e+08,no
3,5203407567,0,0,0,27,0.012920,0,0,0,340,...,0,19,0,0,0.031989,0,4,11511.346939,7.102748e+08,no
4,3944654540,0,0,0,7,0.204396,0,0,0,371,...,0,3,0,0,0.055030,0,38,26378.770833,7.995144e+08,yes


In [108]:
raw_file.describe()

,id,figure_RRT,average_comm,figure_at_sigle,hashtag,reputation,late_night_times,followee_num,follow_ratio,post_num,figure_at,active_day_ratio,mention,day_interval_variance,figure_at_every,test_similarity,text_similarity,url,time_interval_mean,time_interval_var
count,1.566000e+03,1566.0,1566.0,1566.0,1566.000000,1566.000000,1566.0,1566.0,1566.0,1566.000000,1566.0,1566.0,1566.000000,1566.0,1566.0,1566.000000,1566.0,1566.000000,1566.000000,1.566000e+03
mean,2.967337e+09,0.0,0.0,0.0,10.814815,2.009261,0.0,0.0,0.0,1460.061941,0.0,0.0,6.033206,0.0,0.0,0.062515,0.0,7.306513,29051.849704,7.228954e+08
std,1.184598e+09,0.0,0.0,0.0,9.773090,28.911290,0.0,0.0,0.0,3472.917546,0.0,0.0,7.110370,0.0,0.0,0.091577,0.0,10.694027,10882.851825,2.586876e+08
min,1.005384e+09,0.0,0.0,0.0,0.000000,0.009146,0.0,0.0,0.0,2.000000,0.0,0.0,0.000000,0.0,0.0,0.020051,0.0,0.000000,6.204082,0.000000e+00
25%,2.021067e+09,0.0,0.0,0.0,3.000000,0.317523,0.0,0.0,0.0,294.750000,0.0,0.0,1.000000,0.0,0.0,0.030094,0.0,1.000000,22165.185566,6.211617e+08
50%,2.727800e+09,0.0,0.0,0.0,8.000000,0.579435,0.0,0.0,0.0,669.000000,0.0,0.0,4.000000,0.0,0.0,0.037440,0.0,3.000000,30547.811437,7.618443e+08
75%,3.540640e+09,0.0,0.0,0.0,16.000000,1.051232,0.0,0.0,0.0,1520.750000,0.0,0.0,8.000000,0.0,0.0,0.055187,0.0,8.000000,37380.515306,8.798472e+08
max,5.397886e+09,0.0,0.0,0.0,49.000000,1129.000000,0.0,0.0,0.0,102353.000000,0.0,0.0,50.000000,0.0,0.0,1.000000,0.0,50.000000,65396.666667,1.731597e+09


In [110]:
aval_feat_list = ['hashtag', 'reputation', 'post_num', 'test_similarity', 'url', 'time_interval_mean',
                 'time_interval_var']
data = raw_file[aval_feat_list]
target = raw_file[['is_spammer']]

In [111]:
from sklearn.model_selection import GridSearchCV
target['is_spammer']=target['is_spammer'].map(dict(yes=1, no=0))

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


# 2. Machine Learning Model

## 2.1 Classical Models

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

In [5]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)
abc = AdaBoostClassifier(n_estimators=62, random_state=111)
bc = BaggingClassifier(n_estimators=9, random_state=111)
etc = ExtraTreesClassifier(n_estimators=9, random_state=111)

In [25]:
clfs = {'SVC' : svc,'KNeighborsClassifier' : knc, 'MultinomialNaiveBayes': mnb, 'DecisionTreeClassifier': dtc, 
        'LogisticRegression': lrc, 'RandomForestClassifier': rfc, 'AdaBoost': abc, 'BaggingClassifier': bc, 
        'ExtraTreesClassifier': etc}

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5)

# from sklearn.model_selection import ShuffleSplit
# cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
from sklearn.utils import column_or_1d
target = column_or_1d(target, warn=True)
pred_scores = []
for k,v in clfs.items():
    results = cross_val_score(v, data, target, cv=cv)
    print(k, results, results.mean())
    pred_scores.append((k, results))

BaggingClassifier [ 0.72929936  0.68690096  0.69648562  0.69648562  0.68370607] 0.698575527564
KNeighborsClassifier [ 0.62420382  0.65175719  0.62939297  0.61980831  0.5686901 ] 0.618770476791
RandomForestClassifier [ 0.72611465  0.73801917  0.70607029  0.73801917  0.68690096] 0.719024846869
LogisticRegression [ 0.72292994  0.74440895  0.67731629  0.69968051  0.61980831] 0.692828798763
AdaBoost [ 0.73566879  0.72523962  0.69648562  0.69968051  0.69009585] 0.709434077451
ExtraTreesClassifier [ 0.72292994  0.71246006  0.69648562  0.63897764  0.67731629] 0.689633910584
MultinomialNaiveBayes [ 0.57643312  0.57188498  0.5942492   0.5942492   0.56549521] 0.580462343054
DecisionTreeClassifier [ 0.68789809  0.63578275  0.61980831  0.63258786  0.59105431] 0.633426263202
SVC [ 0.47133758  0.52715655  0.50479233  0.52715655  0.50159744] 0.506408091003


In [16]:
df = pd.DataFrame.from_items(pred_scores,orient='index', columns=['Score'])
df

,Score
DT,0.687898
BgC,0.729299
NB,0.576433
LR,0.722930
AdaBoost,0.735669
RF,0.726115
KN,0.624204
ETC,0.722930
SVC,0.471338


## 2.2 XGBoost

In [1]:
# split to train and test sets
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(
    data, target, test_size=0.33, random_state=0)

NameError: name 'data' is not defined

In [8]:
train_X = train_X.as_matrix()
test_X = test_X.as_matrix()
train_y = train_y.as_matrix()

In [9]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)
gbm.fit(train_X, train_y)
predictions = gbm.predict(test_X)

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 70.41%


In [11]:
# or use 10 fold cross validation
from sklearn.model_selection import cross_val_score
model = xgb.XGBClassifier()

from sklearn.model_selection import KFold
cv = KFold(n_splits=5)

# from sklearn.model_selection import ShuffleSplit
# cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)

results = cross_val_score(model, data, target, cv=cv)
print(results)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# initial mean accuracy 48.54%, deviation 31.44%

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array w

[ 0.7388535   0.74440895  0.73162939  0.73801917  0.67731629]
Accuracy: 72.60% (2.47%)


/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array w

# 3. Parameter Tuning 

## 3.1 xgboost model parameters

In [117]:
def cross_val(alg):
#     kfold = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
    kfold = KFold(n_splits=5, random_state=0)
    scoring = ['accuracy', 'recall']
    results = cross_val_score(alg, data, target, cv=kfold, scoring='accuracy')
    print(results)
    print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    # initial mean accuracy 48.54%, deviation 31.44%

### 3.1.1. Find the number of estimators for a high learning rate

In [116]:
from xgboost.sklearn import XGBClassifier
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
cross_val(xgb1)

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array w

[ 0.69426752  0.76433121  0.67515924  0.71974522  0.65605096  0.7388535
  0.71153846  0.71794872  0.69871795  0.66666667]
Accuracy: 70.43% (3.16%)


In [77]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}


from sklearn.model_selection import ShuffleSplit
shuffle = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)

gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=True, cv=shuffle)
gsearch1.fit(data,target)

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array w

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'max_depth': range(3, 10, 2), 'min_child_weight': range(1, 6, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [78]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.79479, std: 0.01020, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.79241, std: 0.00427, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.79536, std: 0.00753, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.78463, std: 0.00763, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.78323, std: 0.00436, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.78545, std: 0.00607, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.78021, std: 0.00583, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.78295, std: 0.00264, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.78003, std: 0.00899, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.78133, std: 0.00423, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.78033, std: 0.00538, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.78338, std: 0.00806, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 5

In [81]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test2 = {
    'max_depth':[2,3,4],
    'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=3,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch2.fit(data,target)

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array w

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=5, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'max_depth': [2, 3, 4], 'min_child_weight': [4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [82]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.80417, std: 0.01210, params: {'max_depth': 2, 'min_child_weight': 4},
  mean: 0.80557, std: 0.01090, params: {'max_depth': 2, 'min_child_weight': 5},
  mean: 0.80541, std: 0.01006, params: {'max_depth': 2, 'min_child_weight': 6},
  mean: 0.79862, std: 0.01026, params: {'max_depth': 3, 'min_child_weight': 4},
  mean: 0.79536, std: 0.00753, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.80027, std: 0.00964, params: {'max_depth': 3, 'min_child_weight': 6},
  mean: 0.78868, std: 0.00650, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.79109, std: 0.00816, params: {'max_depth': 4, 'min_child_weight': 5},
  mean: 0.79032, std: 0.00704, params: {'max_depth': 4, 'min_child_weight': 6}],
 {'max_depth': 2, 'min_child_weight': 5},
 0.8055739087901318)

In [84]:
param_test2b = {
    'min_child_weight':[6,8,10,12, 14, 16]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch2b.fit(data,target)

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array w

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=5, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'min_child_weight': [6, 8, 10, 12, 14, 16]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [85]:
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.80541, std: 0.01006, params: {'min_child_weight': 6},
  mean: 0.80370, std: 0.01083, params: {'min_child_weight': 8},
  mean: 0.80226, std: 0.01086, params: {'min_child_weight': 10},
  mean: 0.80187, std: 0.01239, params: {'min_child_weight': 12},
  mean: 0.80064, std: 0.01296, params: {'min_child_weight': 14},
  mean: 0.80053, std: 0.01450, params: {'min_child_weight': 16}],
 {'min_child_weight': 6},
 0.80540973767667989)

In [88]:
param_test2c = {
    'min_child_weight':[16, 20, 24, 28, 32, 36, 40]
}
gsearch2c = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2c, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch2c.fit(data,target)

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array w

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'min_child_weight': [16, 20, 24, 28, 32, 36, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [89]:
gsearch2c.grid_scores_, gsearch2c.best_params_, gsearch2c.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.80053, std: 0.01450, params: {'min_child_weight': 16},
  mean: 0.80018, std: 0.01497, params: {'min_child_weight': 20},
  mean: 0.79912, std: 0.01512, params: {'min_child_weight': 24},
  mean: 0.79850, std: 0.01580, params: {'min_child_weight': 28},
  mean: 0.79884, std: 0.01652, params: {'min_child_weight': 32},
  mean: 0.79619, std: 0.01642, params: {'min_child_weight': 36},
  mean: 0.79626, std: 0.01607, params: {'min_child_weight': 40}],
 {'min_child_weight': 16},
 0.80052918244581583)

In [92]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch3.fit(data,target)

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array w

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [93]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.80541, std: 0.01006, params: {'gamma': 0.0},
  mean: 0.80654, std: 0.00929, params: {'gamma': 0.1},
  mean: 0.80654, std: 0.00929, params: {'gamma': 0.2},
  mean: 0.80654, std: 0.00929, params: {'gamma': 0.3},
  mean: 0.80702, std: 0.00892, params: {'gamma': 0.4}],
 {'gamma': 0.4},
 0.80701733827438604)

In [ ]:
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=7,
        min_child_weight=36,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
# modelfit(xgb2, data)
cross_val(xgb2, data)

### Tune subsample and colsample_bytree

In [94]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=2,
                                        min_child_weight=6, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch4.fit(data,target)

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array w

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=177, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [95]:
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.80316, std: 0.01009, params: {'subsample': 0.6, 'colsample_bytree': 0.6},
  mean: 0.80330, std: 0.00938, params: {'subsample': 0.7, 'colsample_bytree': 0.6},
  mean: 0.80263, std: 0.00809, params: {'subsample': 0.8, 'colsample_bytree': 0.6},
  mean: 0.80217, std: 0.00920, params: {'subsample': 0.9, 'colsample_bytree': 0.6},
  mean: 0.80021, std: 0.00945, params: {'subsample': 0.6, 'colsample_bytree': 0.7},
  mean: 0.80107, std: 0.00866, params: {'subsample': 0.7, 'colsample_bytree': 0.7},
  mean: 0.80358, std: 0.00906, params: {'subsample': 0.8, 'colsample_bytree': 0.7},
  mean: 0.80295, std: 0.00843, params: {'subsample': 0.9, 'colsample_bytree': 0.7},
  mean: 0.80341, std: 0.00797, params: {'subsample': 0.6, 'colsample_bytree': 0.8},
  mean: 0.80073, std: 0.00973, params: {'subsample': 0.7, 'colsample_bytree': 0.8},
  mean: 0.80467, std: 0.00996, params: {'subsample': 0.8, 'colsample_bytree': 0.8},
  mean: 0.80155, std: 0.00475, params: {'subsample': 0.9, 'colsample_bytree'

### regularization

In [113]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=2,
                                        min_child_weight=6, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch6.fit(data,target)

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array w

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=177, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'reg_alpha': [1e-05, 0.01, 0.1, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [114]:
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.80663, std: 0.00589, params: {'reg_alpha': 1e-05},
  mean: 0.80581, std: 0.00656, params: {'reg_alpha': 0.01},
  mean: 0.80677, std: 0.00759, params: {'reg_alpha': 0.1},
  mean: 0.80537, std: 0.00733, params: {'reg_alpha': 1},
  mean: 0.56759, std: 0.09559, params: {'reg_alpha': 100}],
 {'reg_alpha': 0.1},
 0.80676985256151657)

In [112]:
from sklearn import cross_validation
xgb = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=2,
                    min_child_weight=6, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
                    objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)
kf = cross_validation.KFold(len(target), n_folds=5)
for train_index, test_index in kf:

   X_train, X_test = data[train_index], data[test_index]
   y_train, y_test = target[train_index], target[test_index]

   xgb.fit(X_train, y_train)
   print(confusion_matrix(y_test, xgb.predict(X_test)))

KeyError: '[ 314  315  316 ..., 1563 1564 1565] not in index'

In [76]:
target['is_spammer'].value_counts()
data.describe()

,hashtag,reputation,post_num,mention,test_similarity,url,time_interval_mean,time_interval_var
count,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1.566000e+03
mean,10.814815,2.009261,1460.061941,6.033206,0.062515,7.306513,29051.849704,7.228954e+08
std,9.773090,28.911290,3472.917546,7.110370,0.091577,10.694027,10882.851825,2.586876e+08
min,0.000000,0.009146,2.000000,0.000000,0.020051,0.000000,6.204082,0.000000e+00
25%,3.000000,0.317523,294.750000,1.000000,0.030094,1.000000,22165.185566,6.211617e+08
50%,8.000000,0.579435,669.000000,4.000000,0.037440,3.000000,30547.811437,7.618443e+08
75%,16.000000,1.051232,1520.750000,8.000000,0.055187,8.000000,37380.515306,8.798472e+08
max,49.000000,1129.000000,102353.000000,50.000000,1.000000,50.000000,65396.666667,1.731597e+09


In [74]:
param_test = {
    'gamma':np.linspace(0.05, 1, num=20),
    'kernel':['rbf', 'poly', 'sigmoid']
}
gsearch = GridSearchCV(estimator = SVC(kernel='sigmoid', gamma=1), 
                       param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch.fit(data, target['is_spammer'])
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/lzc/Common/anaconda3/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.5/site-packages/ipykernel/__main__.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/home/lzc/Common/anaconda